In [2]:
import sys, os, inspect
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from dataset.Dataset import Dataset
from sklearn.metrics import accuracy_score
import torch
from kan.KAN import KAN
from kan.DropKAN import DropKAN
from kan.DropKANLayer import DropKANLayer
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd 

def set_training_mode(module, mode):
    """Set the training mode for a module and all its sub-modules."""
    module.training = mode
    for submodule in module.children():
        set_training_mode(submodule, mode)

In [3]:
work_dir = 'uci datasets/Shuffled/'
log_folder =  'log neuron fun\\'

header=None
file_name, sep, label = 'german.data-numeric', ',', -1
#file_name, sep, label = 'abalone.data', ',', -1
#file_name, sep, label = 'dermatology.data', ',', -1
#file_name, sep, label = 'car.data', ',', -1
file_name, sep, label = 'adult.data', ',', -1
#file_name, sep, label = 'census-income.data', ',', -1
#file_name, sep, label = 'diabetic_data.csv', ',', -1
#file_name, sep, label, header = 'bank-full.csv', ',', -1, 0
#file_name, sep, label = 'connect-4.data', ',', -1
#file_name, sep, label = 'semeion.data', ',', -1

 




data = Dataset(work_dir+file_name, sep, label, divide_dataset=False, header=header)

In [151]:
classifier = RandomForestClassifier(n_estimators=111, random_state=0, n_jobs=-1)
data.divide_dataset(classifier,
                                      normalize=True, 
                                      shuffle=False, 
                                      all_features=True, 
                                      all_instances=True, 
                                      evaluate=True, 
                                      partial_sample=False,
                                      metric=accuracy_score)

In [152]:
data.get_validation_accuracy()

0.850583538083538

In [153]:
data.get_test_accuracy()

0.8527560264087211

In [154]:
torch.round(torch.sigmoid(model(dataset['val_input'])[:,0]))

tensor([1., 1., 1.,  ..., 0., 1., 1.], grad_fn=<RoundBackward0>)

In [155]:
dataset = {}
dataset['train_input'] = torch.from_numpy(data.X_train).float()
dataset['test_input'] = torch.from_numpy(data.X_test).float()
dataset['val_input'] = torch.from_numpy(data.X_val).float()
dataset['train_label'] = torch.from_numpy(data.y_train[:, None])
dataset['test_label'] = torch.from_numpy(data.y_test[:, None])
dataset['val_label'] = torch.from_numpy(data.y_val[:, None])

In [206]:
for j in range(1):
    model = DropKAN(seed=j,width=[data.X_train.shape[1], 10, 1],
                            symbolic_enabled=False, drop_rate=[.0, .0], drop_mode='postact', drop_scale=True, neuron_fun=['sum', 'sum'])
    results = model.train(dataset, opt="Adam", steps=2000, batch=32, lr=0.01, update_grid=False)
    set_training_mode(model, False)
    
    val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float())
    test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float())
    print(0.0, 0.0, 'kan', None, val_acc, test_acc)

description: 100%|█████████████████████████████████████████████| 2000/2000 [00:12<00:00, 155.40it/s]


0.0 0.0 kan None tensor(0.8478) tensor(0.8489)


In [65]:

# Log file name
log_file =  log_folder+file_name+'_log.xlsx'

# DataFrame to store logs
log_df = pd.DataFrame(columns=['neuron_func1', 'neuron_func2', 'drop_rate1', 'drop_rate2', 'drop_mode', 'scale', 'validation_accuracy', 'test_accuracy'])




drop_rate1 = 0
drop_rate2 = 0
drop_mode = None
drop_scale = None

best = 0
for _ in range(1):  # Number of random configurations to test
    random.seed()
    for neuron_func1 in ['sum', 'min', 'max', 'multiply', 'mean', 'std', 'var', 'median', 'norm']:
        for neuron_func2 in ['sum', 'min', 'max', 'multiply', 'mean', 'std', 'var', 'median', 'norm']:
            
            
            model = DropKAN(width=[data.X_train.shape[1], 10,1], grid=3, k=3, bias_trainable=True, sp_trainable=True, sb_trainable=True,
                            symbolic_enabled=False, drop_rate=[drop_rate1, drop_rate2], drop_mode=drop_mode, drop_scale=drop_scale, 
                            neuron_fun=[neuron_func1, neuron_func2])

            results = model.train(dataset, opt="Adam", steps=2000, log=100, batch=32, lr=0.01, update_grid=False)
            set_training_mode(model, False)
            
            val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float())
            test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float())
            
            
                
            new_row = [neuron_func1, neuron_func2,drop_rate1,drop_rate2,drop_mode,drop_scale,val_acc.item(),test_acc.item()]
            
            log_df.loc[len(log_df)] = new_row
        
            # Save the DataFrame to an Excel file
            log_df.to_excel(log_file, index=False)
            print(val_acc, test_acc)
            print(len(log_df))
            print()

print(f"Log saved to {log_file}")

description: 100%|██████████████████████████████████████████████| 2000/2000 [00:25<00:00, 78.96it/s]


tensor(0.9937) tensor(0.9843)
1



description: 100%|██████████████████████████████████████████████| 2000/2000 [00:26<00:00, 76.87it/s]


tensor(0.9937) tensor(0.9937)
2



description: 100%|██████████████████████████████████████████████| 2000/2000 [00:25<00:00, 77.30it/s]


...

tensor(1.) tensor(1.)
79



description: 100%|██████████████████████████████████████████████| 2000/2000 [00:23<00:00, 86.58it/s]


tensor(1.) tensor(1.)
80



description: 100%|██████████████████████████████████████████████| 2000/2000 [00:22<00:00, 89.30it/s]


tensor(1.) tensor(1.)
81

Log saved to log neuron fun\semeion.data_log.xlsx


In [50]:
log_df.to_excel(log_file, index=False)


In [45]:
dropmode = 4
dropscale = True


100*log_df[(log_df['dropmode'] == dropmode) & (log_df['scale'] == dropscale)].sort_values('total_accuracy', ascending=False).iloc[:5]['total_accuracy'].median()

60.5743408203125

In [46]:
100*log_df[(log_df['dropmode'] == dropmode) & (log_df['scale'] == dropscale)].sort_values('total_accuracy', ascending=False).iloc[:5]['total_accuracy'].std()

1.9219389762676125

In [48]:
setting = log_df[(log_df['dropmode'] == dropmode) & (log_df['scale'] == dropscale)].sort_values('validation_accuracy', ascending=False).iloc[:1]

In [ ]:
grouped = log_df.groupby(['dropmode', 'scale'])
best_settings = grouped.apply(lambda x: x.loc[x['validation_accuracy'].idxmax()])

results_df = pd.DataFrame(columns=['dropmode', 'dropscale', 'validation_accuracy', 'test_accuracy'])

for i, (index, setting) in enumerate(best_settings.iterrows()):
    for j in range(5):
        model = KAN(
        seed=j,
        width=[dataset['train_input'].shape[1], 10, 1],
        grid=3,
        k=3,
        symbolic_enabled=False,
        drop_prob=[setting['drop_prob1'], setting['drop_prob2']],
        dropmode=[setting['dropmode'], setting['dropmode']],
        dropscale=setting['scale'],
        neuron_func='sum'
        )
        results = model.train(dataset, opt="Adam", steps=2000, batch=32, lr=0.01)
        set_training_mode(model, False)
        
        val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float()).item()
        test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float()).item()
        print(setting['dropmode'], setting['scale'], val_acc, test_acc)
        results_df.loc[len(results_df)] = [setting['dropmode'], setting['scale'], val_acc, test_acc]
    

In [ ]:
for j in range(5):
        model = KAN(
        seed=j,
        width=[dataset['train_input'].shape[1], 10, 1],
        grid=3,
        k=3,
        symbolic_enabled=False,
        drop_prob=[0.0, 0.0],
        dropmode=[-1, -1],
        dropscale= -1,
        )
        results = model.train(dataset, opt="Adam", steps=2000, batch=32, lr=0.01)
        set_training_mode(model, False)
        
        val_acc = torch.mean((torch.round(model(dataset['val_input'])[:,0]) == dataset['val_label'][:,0]).float()).item()
        test_acc = torch.mean((torch.round(model(dataset['test_input'])[:,0]) == dataset['test_label'][:,0]).float()).item()
        print(setting['dropmode'], setting['scale'], val_acc, test_acc)
        results_df.loc[len(results_df)] = [-1, -1, val_acc, test_acc]

In [ ]:
results_df.to_excel(folder+file_name+'_results.xlsx', index=False) 

In [68]:
results_df

,dropmode,dropscale,validation_accuracy,test_accuracy
0,0.0,False,0.993711,0.990596
1,0.0,False,0.993711,0.978056
2,0.0,False,0.993711,0.987461
3,0.0,False,0.987421,0.987461
4,0.0,False,0.984277,0.990596
5,0.0,True,0.993711,0.984326
6,0.0,True,0.990566,0.981191
7,0.0,True,0.987421,0.981191
8,0.0,True,0.993711,0.987461
9,0.0,True,0.990566,0.981191


In [21]:
import glob


# Step 1: Read all Excel files in the folder
all_files = glob.glob(log_folder + "/*.xlsx")

# Step 2: Initialize an empty DataFrame to store all ranks
all_ranks = pd.DataFrame()

# Step 3: Process each file
for file in all_files:
    # Read the Excel file
    log_df = pd.read_excel(file)
    
    # Calculate the sum of validation_accuracy and test_accuracy
    log_df['val_test_sum'] = log_df['validation_accuracy'] + log_df['test_accuracy']
    
    # Rank combinations by the sum of validation_accuracy and test_accuracy within each file
    log_df['val_test_rank'] = log_df['val_test_sum'].rank(ascending=False, method='min')
    
    # Extract only the necessary columns for ranking
    ranks_df = log_df[['neuron_func1', 'neuron_func2', 'val_test_rank']]

    tmp_df = ranks_df.groupby(['neuron_func1', 'neuron_func2'])['val_test_rank'].median().reset_index()
    filtered_df = tmp_df[tmp_df['neuron_func1'] == tmp_df['neuron_func2']]

    print(file)

    print(filtered_df.sort_values(by='val_test_rank').head(10))

    print()
    
    # Append the ranking results to all_ranks DataFrame
    all_ranks = pd.concat([all_ranks, ranks_df], ignore_index=True)

# Step 4: Calculate the average rank and standard deviation for each combination
average_ranks = all_ranks.groupby(['neuron_func1', 'neuron_func2'])['val_test_rank'].agg(['mean', 'std']).reset_index()

# Rename the columns to reflect that they are the average and standard deviation ranks
average_ranks.rename(columns={'mean': 'average_val_test_rank', 'std': 'std_val_test_rank'}, inplace=True)


# Print the resulting DataFrame
#print(average_ranks)

# Save to Excel or CSV if needed
average_ranks.to_excel('average_val_test_ranks.xlsx', index=False)
# or
# average_ranks.to_csv('average_val_test_ranks.csv', index=False)

filtered_df = average_ranks[average_ranks['neuron_func1'] == average_ranks['neuron_func2']]

# Print the filtered DataFrame
print(filtered_df.sort_values(by='average_val_test_rank'))

log neuron fun\abalone.data_log.xlsx
   neuron_func1 neuron_func2  val_test_rank
70          sum          sum           16.0
50         norm         norm           20.0
10         mean         mean           31.0
60          std          std           52.0
80          var          var           57.0
20       median       median           61.0
40     multiply     multiply           75.0
30          min          min           76.0
0           max          max           81.0

log neuron fun\adult.data_log.xlsx
   neuron_func1 neuron_func2  val_test_rank
80          var          var            7.0
10         mean         mean           11.0
60          std          std           22.0
70          sum          sum           28.0
50         norm         norm           32.0
20       median       median           61.0
30          min          min           63.0
0           max          max           67.0
40     multiply     multiply           73.0

log neuron fun\bank-full.csv_log.xlsx
   neuro

In [15]:
filtered_df = average_ranks[average_ranks['neuron_func1'] == average_ranks['neuron_func2']]

# Print the filtered DataFrame
print(filtered_df.sort_values(by='average_val_test_rank'))

   neuron_func1 neuron_func2  average_val_test_rank  std_val_test_rank
10         mean         mean                   19.1          14.106342
60          std          std                   20.6          17.257527
70          sum          sum                   25.7          18.257723
50         norm         norm                   26.3          17.153231
80          var          var                   28.5          21.685889
30          min          min                   48.5          24.450631
20       median       median                   57.1          18.988008
0           max          max                   60.5          23.731133
40     multiply     multiply                   64.1          15.423287


In [13]:
print(average_ranks.sort_values(by='average_val_test_rank').head(10))

   neuron_func1 neuron_func2  average_val_test_rank  std_val_test_rank
17         mean          var                   14.9          15.595227
14         mean         norm                   15.1          12.350888
61          std          sum                   15.3          16.606893
16         mean          sum                   17.2          16.212478
15         mean          std                   17.6          14.245467
62          std          var                   18.3          11.633763
10         mean         mean                   19.1          14.106342
55          std         mean                   19.4          17.263964
60          std          std                   20.6          17.257527
59          std         norm                   20.6          12.920268


In [29]:
all_ranks[(all_ranks['neuron_func1'] == 'max') & (all_ranks['neuron_func2'] == 'max')]['val_test_rank'].std()

23.731132856791028

In [40]:
import glob
import pandas as pd
from scipy.stats import ranksums


# Step 1: Read all Excel files in the folder
all_files = glob.glob(log_folder + "/*.xlsx")

# Step 2: Initialize an empty DataFrame to store all ranks
all_ranks = pd.DataFrame()

# Step 3: Process each file
for file in all_files:
    # Read the Excel file
    log_df = pd.read_excel(file)
    
    # Calculate the sum of validation_accuracy and test_accuracy
    log_df['val_test_sum'] = log_df['validation_accuracy'] + log_df['test_accuracy']
    
    # Rank combinations by the sum of validation_accuracy and test_accuracy within each file
    log_df['val_test_rank'] = log_df['val_test_sum'].rank(ascending=False, method='min')
    
    # Extract only the necessary columns for ranking
    ranks_df = log_df[['neuron_func1', 'neuron_func2', 'val_test_rank']]
    
    # Append the ranking results to all_ranks DataFrame
    all_ranks = pd.concat([all_ranks, ranks_df], ignore_index=True)

# Step 4: Calculate the average rank for each combination
average_ranks = all_ranks.groupby(['neuron_func1', 'neuron_func2'])['val_test_rank'].mean().reset_index()

# Rename the column to reflect that it is the average rank
average_ranks.rename(columns={'val_test_rank': 'average_val_test_rank'}, inplace=True)

# Filter the DataFrame for combinations where both neuron_func1 and neuron_func2 are 'sum'
sum_sum_ranks = all_ranks[(all_ranks['neuron_func1'] == 'sum') & (all_ranks['neuron_func2'] == 'sum')]

# Print the filtered DataFrame
print("Average ranks for combinations where both functions are 'sum':")
print(sum_sum_ranks)

# Step 5: Compare ranks and perform statistical test
comparison_results = []

# Perform comparison with all other combinations
for index, row in average_ranks.iterrows():
    if not (row['neuron_func1'] == 'sum' and row['neuron_func2'] == 'sum'):
        other_func_ranks = all_ranks[(all_ranks['neuron_func1'] == row['neuron_func1']) & 
                                     (all_ranks['neuron_func2'] == row['neuron_func2'])]['val_test_rank']
        
        # Perform Wilcoxon rank-sum test
        stat, p_value = ranksums(sum_sum_ranks['val_test_rank'], other_func_ranks)
        
        # Check if the p-value indicates a significant difference
        if p_value < 0.1:
            comparison_results.append({
                'neuron_func1': row['neuron_func1'],
                'neuron_func2': row['neuron_func2'],
                'average_val_test_rank': row['average_val_test_rank'],
                'p_value': p_value,
                'significant': True
            })
        else:
            comparison_results.append({
                'neuron_func1': row['neuron_func1'],
                'neuron_func2': row['neuron_func2'],
                'average_val_test_rank': row['average_val_test_rank'],
                'p_value': p_value,
                'significant': False
            })

# Convert the results to a DataFrame
comparison_df = pd.DataFrame(comparison_results)

# Print the comparison results
print("Comparison results:")
print(comparison_df)

# Save the comparison results to an Excel file if needed
comparison_df.to_excel('comparison_results.xlsx', index=False)


Average ranks for combinations where both functions are 'sum':
    neuron_func1 neuron_func2  val_test_rank
0            sum          sum           16.0
81           sum          sum           28.0
162          sum          sum           25.0
243          sum          sum            9.0
324          sum          sum            8.0
405          sum          sum           11.0
486          sum          sum           45.0
567          sum          sum           22.0
648          sum          sum           67.0
729          sum          sum           26.0
Comparison results:
   neuron_func1 neuron_func2  average_val_test_rank   p_value  significant
0           max          max                   60.5  0.007285         True
1           max         mean                   40.7  0.041250         True
2           max       median                   49.5  0.053903         True
3           max          min                   59.6  0.001706         True
4           max     multiply                   

In [42]:
comparison_df[comparison_df['significant'] == True].sort_values('average_val_test_rank')

,neuron_func1,neuron_func2,average_val_test_rank,p_value,significant
17,mean,var,14.9,0.088973,True
70,sum,var,37.8,0.058782,True
5,max,norm,40.4,0.075662,True
1,max,mean,40.7,0.041250,True
74,var,min,41.5,0.034294,True
35,min,var,43.1,0.034294,True
34,min,sum,43.5,0.012611,True
32,min,norm,44.8,0.041250,True
30,min,min,48.5,0.041250,True
6,max,std,48.7,0.008151,True
